In [1]:
import sys
sys.path.insert(0, "D:/myPythonLibs")

In [2]:
# import dgl.nn as dglnn
# from dgl import from_networkx
from torch_geometric.utils import from_networkx
import torch.nn as nn
import torch as th
import torch
import torch.nn.functional as F
# import dgl.function as fn
import networkx as nx
import pandas as pd
import socket
import struct
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import pandas as pd

from torch_geometric.nn import MessagePassing
from torch_geometric.nn.models.tgn import TGNMemory
from torch.utils.data import Dataset, DataLoader, IterableDataset
import time
from sklearn.utils import class_weight

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
cols = [
    'FLOW_START_MILLISECONDS', # for TGN, following are same with SAGE
    'IPV4_SRC_ADDR',
    'L4_SRC_PORT',
    'IPV4_DST_ADDR',
    'L4_DST_PORT',
    'PROTOCOL',
    'L7_PROTO',
    'IN_BYTES',
    'OUT_BYTES',
    'IN_PKTS',
    'OUT_PKTS',
    'TCP_FLAGS',
    'FLOW_DURATION_MILLISECONDS',
    'Label',
    'Attack',
]

In [5]:
# data = pd.read_csv('dataset/NF_ToN_IoT/NF-ToN-IoT-v3.csv', usecols=cols)
# data = pd.read_csv('filtered_NF_ToN_IoT_v3_data.csv', usecols=cols)
# data = pd.read_csv('sample03_NF_ToN_IoT_v3_data.csv', usecols=cols)
data = pd.read_csv('sample008_NF_ToN_IoT_v3_data.csv', usecols=cols)

In [6]:
# data.to_csv('filtered_NF_ToN_IoT_v3_data.csv', index=False)

In [7]:
data.head()

,FLOW_START_MILLISECONDS,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,1556028590624,192.168.1.32,41316,192.168.1.180,8652,6,0,44,1,0,0,2,1,0,Benign
1,1556028590626,192.168.1.32,41315,192.168.1.186,1718,6,0,44,1,0,0,2,1,0,Benign
2,1556028590686,192.168.1.32,41315,192.168.1.250,50001,6,0,44,1,0,0,2,1,0,Benign
3,1556028590704,192.168.1.32,41315,192.168.1.169,700,6,0,44,1,40,1,22,1,0,Benign
4,1556028590710,192.168.1.32,41315,192.168.1.194,10180,6,0,44,1,40,1,22,3,0,Benign


In [8]:
data["Attack"].unique()

array(['Benign', 'scanning', 'dos', 'injection', 'ddos', 'password',
       'xss', 'ransomware', 'Backdoor', 'mitm'], dtype=object)

In [9]:
def sample_flows_time_stratified(df: pd.DataFrame, frac: float, n_bins: int, seed: int = 563):

    df = df.copy()
    df['flow_id'] = df[['IPV4_SRC_ADDR','L4_SRC_PORT','IPV4_DST_ADDR','L4_DST_PORT','PROTOCOL']].astype(str).agg('-'.join, axis=1)

    flow_times = (df.groupby('flow_id')['FLOW_START_MILLISECONDS']
                  .min()
                  .reset_index()
                  .rename(columns={'FLOW_START_MILLISECONDS':'flow_start_ms'}))

    flow_times['time_bin'] = pd.qcut(flow_times['flow_start_ms'], q=n_bins, labels=False, duplicates='drop')

    rng = np.random.default_rng(seed)
    sampled_flows = (flow_times.groupby('time_bin')['flow_id']
                        .apply(lambda idx: idx.sample(frac=frac, random_state=seed))
                        .explode()
                        .values)

    df_sampled = (df[df['flow_id'].isin(sampled_flows)]
                  .drop(columns=['flow_id'])
                  .sort_values('FLOW_START_MILLISECONDS')
                  .reset_index(drop=True))
    return df_sampled

In [10]:
# comment this since already get the sampled data
# data = sample_flows_time_stratified(data, frac=0.08, n_bins=30)

In [11]:
print(data["TCP_FLAGS"].unique())

[ 2 22 20  0 31 16 26 18 27 30  6  4 24 17 28 21 29 25 19 23 41]


In [12]:
print(data["TCP_FLAGS"].unique())

[ 2 22 20  0 31 16 26 18 27 30  6  4 24 17 28 21 29 25 19 23 41]


In [13]:
print(data["L7_PROTO"].unique())

[  0  79  10  97  81  77 381   3   5   4 370 167  92  19 112 132  91 131
 150  41 347  51 158  17   7  12  50  13  89 174  88  20 172 111 166  29
 146  85  23 175 391 170  82 139 100  11 383 226 244 264 114 308   2 332
   1 378 161  28  37 164 213 343 222 154   9 243  18 312   8 153  95 247
  78 159  96  14 365 372 229 259 148 248 397 144 237 362 380 260 278 354
 394  60 182  15 377 204 151 138  40 334 379  72 329 351  43 245 367 360
  65 196  35 349  44 374 390 400 209 162 401 335 251 214 190 265 353]


In [14]:
attack_counts = data['Attack'].value_counts()

attack_counts_df = attack_counts.rename_axis('Attack').reset_index(name='Count')
print(attack_counts_df)

       Attack    Count
0      Benign  1406338
1        ddos   333642
2         xss   224967
3    password   125455
4    scanning   108818
5   injection    30551
6    Backdoor    16918
7         dos    16248
8        mitm      344
9  ransomware      340


In [15]:
data

,FLOW_START_MILLISECONDS,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,1556028590624,192.168.1.32,41316,192.168.1.180,8652,6,0,44,1,0,0,2,1,0,Benign
1,1556028590626,192.168.1.32,41315,192.168.1.186,1718,6,0,44,1,0,0,2,1,0,Benign
2,1556028590686,192.168.1.32,41315,192.168.1.250,50001,6,0,44,1,0,0,2,1,0,Benign
3,1556028590704,192.168.1.32,41315,192.168.1.169,700,6,0,44,1,40,1,22,1,0,Benign
4,1556028590710,192.168.1.32,41315,192.168.1.194,10180,6,0,44,1,40,1,22,3,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263616,1556549130829,192.168.1.180,59381,192.168.1.190,53,17,5,70,1,141,1,0,1,0,Benign
2263617,1556549132833,192.168.1.180,45351,192.168.1.190,53,17,5,69,1,123,1,0,1,0,Benign
2263618,1556549132834,192.168.1.180,45351,192.168.1.190,53,17,5,69,1,123,1,0,1,0,Benign
2263619,1556549152834,192.168.1.133,0,224.0.0.251,0,2,82,32,1,0,0,0,1,0,Benign


In [16]:
# data.to_csv('sample008_NF_ToN_IoT_v3_data.csv', index=False)

In [17]:
# data["L7_PROTO"] = data["L7_PROTO"].astype(int)

In [18]:
print(data["L7_PROTO"].unique())

[  0  79  10  97  81  77 381   3   5   4 370 167  92  19 112 132  91 131
 150  41 347  51 158  17   7  12  50  13  89 174  88  20 172 111 166  29
 146  85  23 175 391 170  82 139 100  11 383 226 244 264 114 308   2 332
   1 378 161  28  37 164 213 343 222 154   9 243  18 312   8 153  95 247
  78 159  96  14 365 372 229 259 148 248 397 144 237 362 380 260 278 354
 394  60 182  15 377 204 151 138  40 334 379  72 329 351  43 245 367 360
  65 196  35 349  44 374 390 400 209 162 401 335 251 214 190 265 353]


In [19]:
data

,FLOW_START_MILLISECONDS,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,1556028590624,192.168.1.32,41316,192.168.1.180,8652,6,0,44,1,0,0,2,1,0,Benign
1,1556028590626,192.168.1.32,41315,192.168.1.186,1718,6,0,44,1,0,0,2,1,0,Benign
2,1556028590686,192.168.1.32,41315,192.168.1.250,50001,6,0,44,1,0,0,2,1,0,Benign
3,1556028590704,192.168.1.32,41315,192.168.1.169,700,6,0,44,1,40,1,22,1,0,Benign
4,1556028590710,192.168.1.32,41315,192.168.1.194,10180,6,0,44,1,40,1,22,3,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263616,1556549130829,192.168.1.180,59381,192.168.1.190,53,17,5,70,1,141,1,0,1,0,Benign
2263617,1556549132833,192.168.1.180,45351,192.168.1.190,53,17,5,69,1,123,1,0,1,0,Benign
2263618,1556549132834,192.168.1.180,45351,192.168.1.190,53,17,5,69,1,123,1,0,1,0,Benign
2263619,1556549152834,192.168.1.133,0,224.0.0.251,0,2,82,32,1,0,0,0,1,0,Benign


In [20]:
data['IPV4_SRC_ADDR'] = data.IPV4_SRC_ADDR.apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))

In [21]:
data['IPV4_SRC_ADDR'] = data.IPV4_SRC_ADDR.apply(str)
data['L4_SRC_PORT'] = data.L4_SRC_PORT.apply(str)
data['IPV4_DST_ADDR'] = data.IPV4_DST_ADDR.apply(str)
data['L4_DST_PORT'] = data.L4_DST_PORT.apply(str)

In [22]:
data['IPV4_SRC_ADDR'] = data['IPV4_SRC_ADDR'] + ':' + data['L4_SRC_PORT']
data['IPV4_DST_ADDR'] = data['IPV4_DST_ADDR'] + ':' + data['L4_DST_PORT']

In [23]:
data.drop(columns=['L4_SRC_PORT','L4_DST_PORT'],inplace=True)

In [24]:
data

,FLOW_START_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,1556028590624,172.20.142.138:41316,192.168.1.180:8652,6,0,44,1,0,0,2,1,0,Benign
1,1556028590626,172.17.244.203:41315,192.168.1.186:1718,6,0,44,1,0,0,2,1,0,Benign
2,1556028590686,172.17.158.185:41315,192.168.1.250:50001,6,0,44,1,0,0,2,1,0,Benign
3,1556028590704,172.29.142.170:41315,192.168.1.169:700,6,0,44,1,40,1,22,1,0,Benign
4,1556028590710,172.20.231.27:41315,192.168.1.194:10180,6,0,44,1,40,1,22,3,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263616,1556549130829,172.17.5.215:59381,192.168.1.190:53,17,5,70,1,141,1,0,1,0,Benign
2263617,1556549132833,172.17.162.3:45351,192.168.1.190:53,17,5,69,1,123,1,0,1,0,Benign
2263618,1556549132834,172.26.151.247:45351,192.168.1.190:53,17,5,69,1,123,1,0,1,0,Benign
2263619,1556549152834,172.24.12.58:0,224.0.0.251:0,2,82,32,1,0,0,0,1,0,Benign


In [25]:
data.drop(columns=['Attack'],inplace = True)

In [26]:
data.rename(columns={"Label": "label"},inplace = True)

In [27]:
label = data.label

In [28]:
data.drop(columns=['label'],inplace = True)

In [29]:
scaler = StandardScaler()

In [30]:
data =  pd.concat([data, label], axis=1)

In [31]:
data

,FLOW_START_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,label
0,1556028590624,172.20.142.138:41316,192.168.1.180:8652,6,0,44,1,0,0,2,1,0
1,1556028590626,172.17.244.203:41315,192.168.1.186:1718,6,0,44,1,0,0,2,1,0
2,1556028590686,172.17.158.185:41315,192.168.1.250:50001,6,0,44,1,0,0,2,1,0
3,1556028590704,172.29.142.170:41315,192.168.1.169:700,6,0,44,1,40,1,22,1,0
4,1556028590710,172.20.231.27:41315,192.168.1.194:10180,6,0,44,1,40,1,22,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2263616,1556549130829,172.17.5.215:59381,192.168.1.190:53,17,5,70,1,141,1,0,1,0
2263617,1556549132833,172.17.162.3:45351,192.168.1.190:53,17,5,69,1,123,1,0,1,0
2263618,1556549132834,172.26.151.247:45351,192.168.1.190:53,17,5,69,1,123,1,0,1,0
2263619,1556549152834,172.24.12.58:0,224.0.0.251:0,2,82,32,1,0,0,0,1,0


In [32]:
# X = data.drop(columns=['label'])
# y = data['label']

In [33]:
# X_train, X_test, y_train, y_test = train_test_split(
#      X, y, test_size=0.3, random_state=123,stratify= y)

# normalization of time stamp

# df = data.sort_values('FLOW_START_MILLISECONDS').reset_index(drop=True)
# df['ts_sec'] = df['FLOW_START_MILLISECONDS'] / 1000.0

# t_min = df['ts_sec'].min()
# t_max = df['ts_sec'].max()
# df['ts_norm'] = (df['ts_sec'] - t_min) / (t_max - t_min)

# df = df.drop(columns=['FLOW_START_MILLISECONDS', 'ts_sec'])

def min_max(ts_train: pd.Series, ts_test: pd.Series):
    ts_train = ts_train / 1000.0
    ts_test = ts_test / 1000.0
    
    ts_min, ts_max = ts_train.min(), ts_train.max()
    
    ts_train_norm = (ts_train - ts_min) / (ts_max - ts_min)
    ts_test_norm = (ts_test - ts_min) / (ts_max - ts_min)

    return ts_train_norm, ts_test_norm

def get_sec(ts_train: pd.Series, ts_test: pd.Series):
    ts_train = ts_train / 1000.0
    ts_test = ts_test / 1000.0

    return ts_train, ts_test

In [34]:
df = data.sort_values('FLOW_START_MILLISECONDS').reset_index(drop=True)

split_idx = int(len(df) * 0.7)

# 3. 切分
train_df = df.iloc[:split_idx]
test_df  = df.iloc[split_idx:]

# 4. 拆 X / y
feature_cols = ['IPV4_SRC_ADDR','IPV4_DST_ADDR','PROTOCOL','L7_PROTO','IN_BYTES',
                'IN_PKTS','OUT_BYTES','OUT_PKTS','TCP_FLAGS','FLOW_DURATION_MILLISECONDS']
X_train = train_df[feature_cols]
y_train = train_df['label']
ts_train = train_df['FLOW_START_MILLISECONDS']

X_test  = test_df[feature_cols]
y_test  = test_df['label']
ts_test = test_df['FLOW_START_MILLISECONDS']

In [35]:
ts_train_norm, ts_test_norm = min_max(ts_train, ts_test)
ts_train_sec, ts_test_sec = get_sec(ts_train, ts_test)

In [36]:
X_train

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
0,172.20.142.138:41316,192.168.1.180:8652,6,0,44,1,0,0,2,1
1,172.17.244.203:41315,192.168.1.186:1718,6,0,44,1,0,0,2,1
2,172.17.158.185:41315,192.168.1.250:50001,6,0,44,1,0,0,2,1
3,172.29.142.170:41315,192.168.1.169:700,6,0,44,1,40,1,22,1
4,172.20.231.27:41315,192.168.1.194:10180,6,0,44,1,40,1,22,3
...,...,...,...,...,...,...,...,...,...,...
1584529,172.28.30.154:42656,192.168.1.184:443,6,91,112,2,60,1,18,1
1584530,172.19.31.73:42666,192.168.1.184:443,6,91,112,2,60,1,18,1
1584531,172.24.25.54:42706,192.168.1.184:443,6,91,112,2,60,1,18,1
1584532,172.22.155.241:42950,192.168.1.184:443,6,91,112,2,60,1,18,1


In [37]:
ts_train_norm

0          0.000000e+00
1          8.920699e-09
2          2.765619e-07
3          3.568535e-07
4          3.836177e-07
               ...     
1584529    9.999999e-01
1584530    9.999999e-01
1584531    9.999999e-01
1584532    1.000000e+00
1584533    1.000000e+00
Name: FLOW_START_MILLISECONDS, Length: 1584534, dtype: float64

In [38]:
ts_train_sec

0          1.556029e+09
1          1.556029e+09
2          1.556029e+09
3          1.556029e+09
4          1.556029e+09
               ...     
1584529    1.556253e+09
1584530    1.556253e+09
1584531    1.556253e+09
1584532    1.556253e+09
1584533    1.556253e+09
Name: FLOW_START_MILLISECONDS, Length: 1584534, dtype: float64

In [39]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL'])
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)

X_test = encoder.transform(X_test)

In [40]:
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns ))  - set(list(['label'])) )
X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])


In [41]:
cols_to_norm

['TCP_FLAGS',
 'L7_PROTO',
 'IN_BYTES',
 'OUT_PKTS',
 'OUT_BYTES',
 'PROTOCOL',
 'FLOW_DURATION_MILLISECONDS',
 'IN_PKTS']

In [42]:
X_train

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
0,172.20.142.138:41316,192.168.1.180:8652,0.231468,-1.001833,-0.033983,-0.074776,-0.029981,-0.161224,-0.467613,-0.232999
1,172.17.244.203:41315,192.168.1.186:1718,0.231468,-1.001833,-0.033983,-0.074776,-0.029981,-0.161224,-0.467613,-0.232999
2,172.17.158.185:41315,192.168.1.250:50001,0.231468,-1.001833,-0.033983,-0.074776,-0.029981,-0.161224,-0.467613,-0.232999
3,172.29.142.170:41315,192.168.1.169:700,0.231468,-1.001833,-0.033983,-0.074776,-0.023898,-0.004644,-0.599010,-0.232999
4,172.20.231.27:41315,192.168.1.194:10180,0.231468,-1.001833,-0.033983,-0.074776,-0.023898,-0.004644,-0.599010,-0.225640
...,...,...,...,...,...,...,...,...,...,...
1584529,172.28.30.154:42656,192.168.1.184:443,0.231468,0.709400,-0.025470,-0.042689,-0.020856,-0.004644,1.526800,-0.232999
1584530,172.19.31.73:42666,192.168.1.184:443,0.231468,0.709400,-0.025470,-0.042689,-0.020856,-0.004644,1.526800,-0.232999
1584531,172.24.25.54:42706,192.168.1.184:443,0.231468,0.709400,-0.025470,-0.042689,-0.020856,-0.004644,1.526800,-0.232999
1584532,172.22.155.241:42950,192.168.1.184:443,0.231468,0.709400,-0.025470,-0.042689,-0.020856,-0.004644,1.526800,-0.232999


In [43]:
X_train['h'] = X_train[ cols_to_norm ].values.tolist()
X_test['h'] = X_test[cols_to_norm].values.tolist()

In [44]:
df_edges_train = X_train.copy()
df_edges_train['label'] = y_train

df_edges_test = X_test.copy()
df_edges_test['label'] = y_test

In [45]:
X_train

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,h
0,172.20.142.138:41316,192.168.1.180:8652,0.231468,-1.001833,-0.033983,-0.074776,-0.029981,-0.161224,-0.467613,-0.232999,"[-0.4676130315302281, -1.0018334803077904, -0...."
1,172.17.244.203:41315,192.168.1.186:1718,0.231468,-1.001833,-0.033983,-0.074776,-0.029981,-0.161224,-0.467613,-0.232999,"[-0.4676130315302281, -1.0018334803077904, -0...."
2,172.17.158.185:41315,192.168.1.250:50001,0.231468,-1.001833,-0.033983,-0.074776,-0.029981,-0.161224,-0.467613,-0.232999,"[-0.4676130315302281, -1.0018334803077904, -0...."
3,172.29.142.170:41315,192.168.1.169:700,0.231468,-1.001833,-0.033983,-0.074776,-0.023898,-0.004644,-0.599010,-0.232999,"[-0.5990102403830938, -1.0018334803077904, -0...."
4,172.20.231.27:41315,192.168.1.194:10180,0.231468,-1.001833,-0.033983,-0.074776,-0.023898,-0.004644,-0.599010,-0.225640,"[-0.5990102403830938, -1.0018334803077904, -0...."
...,...,...,...,...,...,...,...,...,...,...,...
1584529,172.28.30.154:42656,192.168.1.184:443,0.231468,0.709400,-0.025470,-0.042689,-0.020856,-0.004644,1.526800,-0.232999,"[1.5268004874894632, 0.7094004642495088, -0.02..."
1584530,172.19.31.73:42666,192.168.1.184:443,0.231468,0.709400,-0.025470,-0.042689,-0.020856,-0.004644,1.526800,-0.232999,"[1.5268004874894632, 0.7094004642495088, -0.02..."
1584531,172.24.25.54:42706,192.168.1.184:443,0.231468,0.709400,-0.025470,-0.042689,-0.020856,-0.004644,1.526800,-0.232999,"[1.5268004874894632, 0.7094004642495088, -0.02..."
1584532,172.22.155.241:42950,192.168.1.184:443,0.231468,0.709400,-0.025470,-0.042689,-0.020856,-0.004644,1.526800,-0.232999,"[1.5268004874894632, 0.7094004642495088, -0.02..."


In [46]:
# G = nx.from_pandas_edgelist(X_train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR", ['h','label'],create_using=nx.MultiGraph())
# G = nx.from_pandas_edgelist(df_edges, "IPV4_SRC_ADDR", "IPV4_DST_ADDR", ['h','label'],create_using=nx.MultiGraph())

# u_train_ids, uniques_u = pd.factorize(X_train['IPV4_SRC_ADDR'])
# v_train_ids, uniques_v = pd.factorize(X_train['IPV4_DST_ADDR'])
# u_test_ids,  _         = pd.factorize(X_test ['IPV4_SRC_ADDR'])
# v_test_ids,  _         = pd.factorize(X_test ['IPV4_DST_ADDR'])

In [47]:
# u_train = torch.tensor(u_train_ids, dtype=torch.long, device=device)
# v_train = torch.tensor(v_train_ids, dtype=torch.long, device=device)
# u_test  = torch.tensor(u_test_ids, dtype=torch.long, device=device)
# v_test  = torch.tensor(v_test_ids, dtype=torch.long, device=device)

In [48]:
src_train_keys = train_df['IPV4_SRC_ADDR'].tolist()
dst_train_keys = train_df['IPV4_DST_ADDR'].tolist()
src_test_keys  = test_df ['IPV4_SRC_ADDR'].tolist()
dst_test_keys  = test_df ['IPV4_DST_ADDR'].tolist()

In [49]:
t_train_norm = torch.tensor(ts_train_norm.values, dtype=torch.float32).unsqueeze(1)
t_test_norm  = torch.tensor(ts_test_norm.values, dtype=torch.float32).unsqueeze(1)

t_train_sec = torch.tensor(ts_train_sec.values, dtype=torch.float32)
t_test_sec  = torch.tensor(ts_test_sec.values, dtype=torch.float32)

In [50]:
h_arr_train = np.stack(df_edges_train['h'].values)
m_train = torch.tensor(h_arr_train, dtype=torch.float32)

h_arr_test = np.stack(df_edges_test['h'].values)
m_test = torch.tensor(h_arr_test, dtype=torch.float32)

In [51]:
y_train = torch.tensor(df_edges_train['label'].values, dtype=torch.long)
y_test  = torch.tensor(df_edges_test['label'].values,  dtype=torch.long)

In [52]:
# G = G.to_directed()

In [53]:
print(torch.__version__)
print(torch.version.cuda)

2.5.1+cu121
12.1


In [54]:
# G = from_networkx(G,edge_attrs=['h','label'] )

##################### change from original file:
##################### use PyG instead of DGL

# data = from_networkx(G)

# edge_attrs = [attr['h'] for _, _, attr in G.edges(data=True)]
# edge_labels = [attr['label'] for _, _, attr in G.edges(data=True)]

# data.edge_attr = torch.tensor(edge_attrs, dtype=torch.float)
# data.edge_label = torch.tensor(edge_labels, dtype=torch.float)

In [55]:
# G.ndata['h'] = th.ones(G.num_nodes(), G.edata['h'].shape[1])

# data.x = torch.ones(data.num_nodes, data.edge_attr.shape[1])

In [56]:
# G.edata['train_mask'] = th.ones(len(G.edata['h']), dtype=th.bool)
# data.train_mask = torch.ones(data.edge_attr.size(0), dtype=torch.bool, device=data.edge_attr.device)

In [57]:
def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()

In [58]:
# class SAGELayer(MessagePassing):
#     def __init__(self, ndim_in, edims, ndim_out, activation):
#         super(SAGELayer, self).__init__(aggr='mean')  # mean aggregation = fn.mean
#         self.W_msg = nn.Linear(ndim_in + edims, ndim_out)     # Eq4
#         self.W_apply = nn.Linear(ndim_in + ndim_out, ndim_out) # Eq5
#         self.activation = activation

#     def forward(self, x, edge_index, edge_attr):
#         # x: [num_nodes, ndim_in]
#         # edge_attr: [num_edges, edims]
#         return self.propagate(edge_index, x=x, edge_attr=edge_attr)

#     def message(self, x_j, edge_attr):
#         # x_j: source node features for each edge
#         # edge_attr: edge features
#         return self.W_msg(torch.cat([x_j, edge_attr], dim=-1))  # Eq4

#     def update(self, aggr_out, x):
#         # aggr_out: aggregated message per node
#         h_new = self.W_apply(torch.cat([x, aggr_out], dim=-1))  # Eq5
#         return self.activation(h_new)


# class SAGE(nn.Module):
#     def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
#         super(SAGE, self).__init__()
#         self.layers = nn.ModuleList()
#         self.layers.append(SAGELayer(ndim_in, edim, 128, activation))
#         self.layers.append(SAGELayer(128, edim, ndim_out, activation))
#         self.dropout = nn.Dropout(p=dropout)

#     def forward(self, x, edge_index, edge_attr):
#         for i, layer in enumerate(self.layers):
#             if i != 0:
#                 x = self.dropout(x)
#             x = layer(x, edge_index, edge_attr)
#         return x

In [59]:
class TimeEncode(nn.Module):
    def __init__(self, time_dim: int):
        super().__init__()
        self.lin = nn.Linear(1, time_dim)
    def forward(self, t: torch.Tensor) -> torch.Tensor:
        # t: [B,1]
        out = self.lin(t)
        return torch.cat([torch.sin(out), torch.cos(out)], dim=-1)

In [60]:
class SAGELayer(MessagePassing):
    def __init__(self, ndim_in, edims, ndim_out, activation):
        super().__init__(aggr='mean')  # mean aggregation = fn.mean
        self.W_msg = nn.Linear(ndim_in + edims, ndim_out)     # Eq4
        self.W_apply = nn.Linear(ndim_in + ndim_out, ndim_out) # Eq5
        self.activation = activation

    def forward(self, x, edge_index, edge_attr):
        # x: [num_nodes, ndim_in]
        # edge_attr: [num_edges, edims]
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_j, edge_attr):
        # x_j: source node features for each edge
        # edge_attr: edge features
        return self.W_msg(torch.cat([x_j, edge_attr], dim=-1))  # Eq4

    def update(self, aggr_out, x):
        # aggr_out: aggregated message per node
        h_new = self.W_apply(torch.cat([x, aggr_out], dim=-1))  # Eq5
        return self.activation(h_new)

In [61]:
class MLPPredictor(nn.Module):
    def __init__(self, node_dim: int, edge_dim: int, time_dim: int, hidden_dim: int, out_classes: int):
        super().__init__()
        input_dim = 2 * node_dim + edge_dim + 2 * time_dim
        self.lin1 = nn.Linear(input_dim, hidden_dim)
        self.lin2 = nn.Linear(hidden_dim, out_classes)
    def forward(self, h_u: torch.Tensor, h_v: torch.Tensor, e_feat: torch.Tensor, t_enc: torch.Tensor) -> torch.Tensor:
        x = torch.cat([h_u, h_v, e_feat, t_enc], dim=-1)
        x = F.relu(self.lin1(x))
        return self.lin2(x)

In [62]:
# class Model(nn.Module):
#     def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
#         super().__init__()
#         self.gnn = SAGE(ndim_in, ndim_out, edim, activation, dropout)
#         self.pred = MLPPredictor(ndim_out, 2)
#     def forward(self, g, nfeats, efeats):
#         h = self.gnn(g, nfeats, efeats)
#         return self.pred(g, h)

In [63]:
# class Model(nn.Module):
#     def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
#         super().__init__()
#         self.gnn = SAGE(ndim_in, ndim_out, edim, activation, dropout)
#         self.pred = MLPPredictor(ndim_out, 2)

#     def forward(self, data):
#         node_embeddings = self.gnn(data.x, data.edge_index, data.edge_attr)
#         edge_logits = self.pred(node_embeddings, data.edge_index)
#         return edge_logits

In [64]:
class TemporalEGraphSAGE(nn.Module):
    def __init__(self,
                 num_nodes: int,
                 in_dim: int,
                 edge_feat_dim: int,
                 time_dim: int,
                 hidden_dim: int,
                 out_classes: int,
                 dropout: float = 0.1,
                 activation=F.relu):
        super().__init__()

        self.mem = nn.Embedding(num_nodes, hidden_dim)
        nn.init.zeros_(self.mem.weight)

        self.time_enc = TimeEncode(time_dim)

        self.sage1 = SAGELayer(hidden_dim, edge_feat_dim + 2 * time_dim, hidden_dim, activation)
        self.sage2 = SAGELayer(hidden_dim, edge_feat_dim + 2 * time_dim, hidden_dim, activation)
        self.dropout = nn.Dropout(dropout)

        self.pred = MLPPredictor(hidden_dim, edge_feat_dim, time_dim, hidden_dim, out_classes)
        
    def forward(self,
                src: torch.LongTensor,
                dst: torch.LongTensor,
                e_feat: torch.Tensor,
                ts: torch.Tensor) -> torch.Tensor:
        # src,dst: [B], e_feat: [B, edge_feat_dim], ts: [B,1]

        t_enc = self.time_enc(ts)  # [B,2*time_dim]

        x = self.mem.weight       # [num_nodes, hidden_dim]

        edge_attr = torch.cat([e_feat, t_enc], dim=-1)  # [B, edge_dim+2*time_dim]

        edge_index = torch.stack([src, dst], dim=0)

        h = self.sage1(x, edge_index, edge_attr)
        h = self.dropout(h)
        h = self.sage2(h, edge_index, edge_attr)
        h = self.dropout(h)

        h_u = h[src]
        h_v = h[dst]

        all_idx = torch.cat([src, dst], dim=0)
        all_state = torch.cat([h_u, h_v], dim=0)
        with torch.no_grad():
            self.mem.weight.index_copy_(0, all_idx, all_state)

        return self.pred(h_u, h_v, e_feat, t_enc)

In [65]:
# from sklearn.utils import class_weight
# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(G.edata['label'].cpu().numpy()),
#                                                  G.edata['label'].cpu().numpy())



# class_weights = class_weight.compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(data.edge_label.cpu().numpy()),
#     y=data.edge_label.cpu().numpy()
# )

# class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
# class_weights = torch.tensor(class_weights, dtype=torch.float, device = device)

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train.cpu().numpy()),
    y = y_train.cpu().numpy()
)
class_weights = torch.tensor(class_weights, dtype=torch.float, device=device)

In [66]:
# class_weights = th.FloatTensor(class_weights).cuda()
criterion = nn.CrossEntropyLoss(weight = class_weights)

In [67]:
# G = G.to('cuda:0')
# G.device

In [68]:
# G.ndata['h'].device

In [69]:
# G.edata['h'].device

In [70]:
# data = data.to(device)
# t_train = t_train.to(device)
# m_train = m_train.to(device)
# y_train = y_train.to(device)

In [71]:
# torch.cuda.empty_cache()

In [73]:
## map use for training speed
all_keys = pd.concat([train_df['IPV4_SRC_ADDR'], train_df['IPV4_DST_ADDR']]).unique()
pre_id_map = {key: idx for idx, key in enumerate(all_keys)}
num_nodes = len(all_keys)
print(num_nodes)

src_train_idx = [pre_id_map[k] for k in src_train_keys]
dst_train_idx = [pre_id_map[k] for k in dst_train_keys]

src_idx_all = torch.tensor(src_train_idx, dtype=torch.long)
dst_idx_all = torch.tensor(dst_train_idx, dtype=torch.long)

1765866


In [80]:
class TimeWindowDataset(IterableDataset):
    def __init__(self, src_keys, dst_keys, feats, labels, ts, window_sec):
        self.src_keys  = src_keys
        self.dst_keys  = dst_keys
        self.feats     = feats
        self.labels    = labels
        self.ts        = ts.squeeze()
        self.window    = window_sec

    def __iter__(self):
        start_time = self.ts[0].item()
        buffer = []
        for i, t in enumerate(self.ts):
            if t.item() < start_time + self.window:
                buffer.append(i)
            else:
                yield buffer
                buffer = [i]
                start_time = t.item()
        if buffer:
            yield buffer

window_sec = 3600 
train_ds = TimeWindowDataset(
    src_keys = src_train_keys,
    dst_keys = dst_train_keys,
    feats = m_train,
    labels = y_train,
    ts = t_train_sec,
    window_sec = window_sec
)


def collate_time_batch(idxs):
    idxs = torch.tensor(idxs, dtype=torch.long)
    src    = src_idx_all[idxs]
    dst    = dst_idx_all[idxs]
    feats  = m_train[idxs]
    labels = y_train[idxs]
    ts     = t_train_norm[idxs]
    return src, dst, feats, labels, ts

loader = DataLoader(
    train_ds,
    batch_size=None,
    collate_fn=collate_time_batch,
    num_workers=0
)

In [81]:
t0 = time.time()
batch = next(iter(loader))    # 拿第 1 个窗口 batch
t1 = time.time()

print(f"Time to load one time‐window batch: {t1 - t0:.3f} s")

# 可以解包看看形状
src_idx, dst_idx, feats_batch, labels_batch, ts_batch = batch
print("Shapes →",
      "src:", src_idx.shape,
      "dst:", dst_idx.shape,
      "feats:", feats_batch.shape,
      "labels:", labels_batch.shape,
      "ts:", ts_batch.shape)

Time to load one time‐window batch: 8.015 s
Shapes → src: torch.Size([190515]) dst: torch.Size([190515]) feats: torch.Size([190515, 8]) labels: torch.Size([190515]) ts: torch.Size([190515, 1])


In [77]:
# Get the first window of indices
first_window_idxs = next(iter(train_ds))  # This yields a 1D Tensor of indices
print(f"Number of samples in first window: {len(first_window_idxs)}")

# Extract the corresponding times (in seconds)
times = t_train_sec[first_window_idxs]

# Compute min, max, and duration
t_min = times.min().item()
t_max = times.max().item()
duration = t_max - t_min

print(f"Window start time: {t_min:.3f} s")
print(f"Window end   time: {t_max:.3f} s")
print(f"Window duration: {duration:.3f} s")
print(f"Does duration <= window_sec? {'Yes' if duration <= 3600 else 'No'}")

Number of samples in first window: 190515
Window start time: 1556028544.000 s
Window end   time: 1556032128.000 s
Window duration: 3584.000 s
Does duration <= window_sec? Yes


In [78]:
# print("m_train device:", m_train.device, "y_train device:", y_train.device)

m_train device: cpu y_train device: cpu


In [82]:
model = TemporalEGraphSAGE(
    num_nodes    = num_nodes,
    in_dim       = None,
    edge_feat_dim= m_train.shape[1],
    time_dim     = 16,
    hidden_dim   = 128,
    out_classes  = 2,
    dropout      = 0.2,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


num_epochs = 20
for epoch in range(1, num_epochs+1):
    model.train()
    # cold start clear mem
    model.mem.weight.data.zero_()
    
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    
    t_start = torch.cuda.Event(enable_timing=True)
    t_end   = torch.cuda.Event(enable_timing=True)
    t_start.record()
    
    for src_batch, dst_batch, feats_batch, labels_batch, ts_batch in loader:
        src_batch    = src_batch.to(device)
        dst_batch    = dst_batch.to(device)
        feats_batch  = feats_batch.to(device)
        labels_batch = labels_batch.to(device)
        ts_batch     = ts_batch.to(device)
        
        optimizer.zero_grad()
        logits = model(src_batch, dst_batch, feats_batch, ts_batch)  # [B,2]
        loss   = criterion(logits, labels_batch)

        loss.backward()
        optimizer.step()
        
        total_loss   += loss.item() * labels_batch.size(0)
        preds         = logits.argmax(dim=1)
        total_correct+= (preds == labels_batch).sum().item()
        total_samples += labels_batch.size(0)
    
    t_end.record()
    torch.cuda.synchronize()
    epoch_time_s = t_start.elapsed_time(t_end) / 1000.0
    
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples * 100.0
    print(f"Epoch {epoch:02d} | Time: {epoch_time_s:.1f}s | "f"Loss: {avg_loss:.4f} | Acc: {accuracy:.2f}%")

Epoch 01 | Time: 517.8s | Loss: 0.7985 | Acc: 56.22%
Epoch 02 | Time: 514.6s | Loss: 0.6259 | Acc: 76.51%
Epoch 03 | Time: 512.9s | Loss: 0.5240 | Acc: 85.71%
Epoch 04 | Time: 509.2s | Loss: 0.4714 | Acc: 86.23%
Epoch 05 | Time: 508.9s | Loss: 0.4359 | Acc: 85.49%
Epoch 06 | Time: 509.4s | Loss: 0.4259 | Acc: 87.09%
Epoch 07 | Time: 507.9s | Loss: 0.3739 | Acc: 90.13%
Epoch 08 | Time: 509.0s | Loss: 0.3691 | Acc: 89.55%
Epoch 09 | Time: 508.9s | Loss: 0.4069 | Acc: 88.30%
Epoch 10 | Time: 507.7s | Loss: 0.3157 | Acc: 89.91%
Epoch 11 | Time: 507.5s | Loss: 0.3102 | Acc: 90.98%
Epoch 12 | Time: 508.9s | Loss: 0.3421 | Acc: 90.18%
Epoch 13 | Time: 508.1s | Loss: 57.4337 | Acc: 86.91%
Epoch 14 | Time: 515.1s | Loss: 0.3407 | Acc: 90.07%
Epoch 15 | Time: 516.0s | Loss: 0.2660 | Acc: 90.86%
Epoch 16 | Time: 511.8s | Loss: 0.3654 | Acc: 87.76%
Epoch 17 | Time: 516.3s | Loss: 0.3118 | Acc: 90.14%
Epoch 18 | Time: 514.3s | Loss: 0.3850 | Acc: 82.50%
Epoch 19 | Time: 513.2s | Loss: 0.2384 | Acc:

In [83]:
## save the model
import joblib
import json

ts_min, ts_max = t_train_sec.min(), t_train_sec.max()
joblib.dump({"min": ts_min.item(), "max": ts_max.item()}, "saved_model_TGNESAGE_v1/TGN_EGraphSAGE_v1_time_norm.pkl")
torch.save(model.state_dict(), "saved_model_TGNESAGE_v1/TGN_EGraphSAGE_v1_weights.pth")
joblib.dump(encoder, "saved_model_TGNESAGE_v1/TGN_EGraphSAGE_v1_encoder.pkl")
joblib.dump(scaler, "saved_model_TGNESAGE_v1/TGN_EGraphSAGE_v1_scaler.pkl")

['saved_model_TGNESAGE_v1/TGN_EGraphSAGE_v1_scaler.pkl']

In [57]:

torch.cuda.empty_cache()

In [58]:
# X_test = encoder.transform(X_test)
# X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
# X_test['h'] = X_test[cols_to_norm].values.tolist()

In [1]:
X_test

NameError: name 'X_test' is not defined

In [87]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss

In [90]:
# 1) Cold‑start 初始化
model.eval()
model.mem.weight.data.zero_()
model.id_map  = {}
model.next_id = 0

# 2) 构造固定大小批次索引列表
batch_size = 20000
N = len(src_test_keys)
batches = [list(range(i, min(i+batch_size, N)))
           for i in range(0, N, batch_size)]

# 3) 测试循环：逐批动态映射 & 前向
results = []

with torch.no_grad():
    for batch_i, idxs in enumerate(batches, 1):
        # ——— 3.1 动态分配节点 ID ———
        src_idx_list = []
        dst_idx_list = []
        for j in idxs:
            sk = src_test_keys[j]
            if sk not in model.id_map:
                model.id_map[sk] = model.next_id
                model.next_id += 1
            src_idx_list.append(model.id_map[sk])
            dk = dst_test_keys[j]
            if dk not in model.id_map:
                model.id_map[dk] = model.next_id
                model.next_id += 1
            dst_idx_list.append(model.id_map[dk])

        src_idx = torch.tensor(src_idx_list, dtype=torch.long, device=device)
        dst_idx = torch.tensor(dst_idx_list, dtype=torch.long, device=device)

        # ——— 3.2 准备特征张量 ———
        feats_batch = m_test[idxs].to(device)            # [B, F]
        labels_batch= y_test[idxs].to(device)            # [B]
        ts_norm     = t_test_norm[idxs].to(device)       # [B,1]

        # ——— 3.3 前向 & 计算指标 ———
        t0 = time.time()
        logits = model(src_idx, dst_idx, feats_batch, ts_norm)
        batch_time = time.time() - t0
        
        loss   = criterion(logits, labels_batch).item()
        probs  = torch.softmax(logits, dim=1).cpu().numpy()
        preds  = probs.argmax(axis=1)
        true   = labels_batch.cpu().numpy()

        batch_acc  = accuracy_score(true, preds)
        batch_logl = log_loss(true, probs, labels=[0,1])
        batch_cm   = confusion_matrix(true, preds, labels=[0,1])

        tn, fp, fn, tp = batch_cm.ravel()
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0

        results.append({
            "batch":      batch_i,
            "size":       len(idxs),
            "time_s":     batch_time,
            "loss":       loss,
            "accuracy":   batch_acc,
            "log_loss":   batch_logl,
            "fpr":        fpr,
            "confusion_matrix": batch_cm,
        })
        print("Finish batch: ", {batch_i})

Finish batch:  {1}
Finish batch:  {2}
Finish batch:  {3}
Finish batch:  {4}
Finish batch:  {5}
Finish batch:  {6}
Finish batch:  {7}
Finish batch:  {8}
Finish batch:  {9}
Finish batch:  {10}
Finish batch:  {11}
Finish batch:  {12}
Finish batch:  {13}
Finish batch:  {14}
Finish batch:  {15}
Finish batch:  {16}
Finish batch:  {17}
Finish batch:  {18}
Finish batch:  {19}
Finish batch:  {20}
Finish batch:  {21}
Finish batch:  {22}
Finish batch:  {23}
Finish batch:  {24}
Finish batch:  {25}
Finish batch:  {26}
Finish batch:  {27}
Finish batch:  {28}
Finish batch:  {29}
Finish batch:  {30}
Finish batch:  {31}
Finish batch:  {32}
Finish batch:  {33}
Finish batch:  {34}


In [91]:
import pandas as pd
df_res = pd.DataFrame(results)
print(df_res[["batch","size","time_s","loss","accuracy","fpr"]])

    batch   size    time_s      loss  accuracy       fpr
0       1  20000  0.006034  0.507888  0.864050  0.248614
1       2  20000  0.006027  1.504003  0.556250  0.306244
2       3  20000  0.005903  0.913534  0.886850  0.208859
3       4  20000  0.005377  0.376659  0.950400  0.348026
4       5  20000  0.005055  0.370379  0.943050  0.405546
5       6  20000  0.006977  0.360555  0.943000  0.399469
6       7  20000  0.004740  0.622729  0.905300  0.645804
7       8  20000  0.005735  0.681178  0.785050  0.726998
8       9  20000  0.005052  0.606104  0.810950  0.307609
9      10  20000  0.003812  0.571193  0.890100  0.684403
10     11  20000  0.005336  0.667158  0.681600  0.918397
11     12  20000  0.005104  0.520696  0.902950  0.750637
12     13  20000  0.004640  0.478026  0.968000  0.416588
13     14  20000  0.006928  0.480622  0.964750  0.434747
14     15  20000  0.004859  0.470976  0.976350  0.429036
15     16  20000  0.007972  0.471601  0.975900  0.404612
16     17  20000  0.005325  0.4

In [110]:
print("Batch 1 CM:\n", results[5]["confusion_matrix"])

Batch 1 CM:
 [[ 1356   902]
 [  238 17504]]


In [104]:
tn_total = fp_total = fn_total = tp_total = 0
for r in results:
    cm = r["confusion_matrix"]
    # ensure shape 2x2
    cm = np.array(cm)
    tn, fp, fn, tp = cm.ravel()
    tn_total += tn
    fp_total += fp
    fn_total += fn
    tp_total += tp

# Compute overall metrics
total_samples = tn_total + fp_total + fn_total + tp_total
overall_accuracy = (tn_total + tp_total) / total_samples
overall_fpr = fp_total / (fp_total + tn_total) if (fp_total + tn_total) > 0 else np.nan
overall_f1 = 2 * tp_total / (2 * tp_total + fp_total + fn_total) if (2 * tp_total + fp_total + fn_total) > 0 else np.nan

# Display results
df_summary = pd.DataFrame([{
    "total_samples": total_samples,
    "accuracy": overall_accuracy,
    "fpr": overall_fpr,
    "f1_score": overall_f1
}])

import matplotlib.pyplot as plt

df_summary

,total_samples,accuracy,fpr,f1_score
0,679087,0.780099,0.378958,0.821853


In [111]:
m_train.shape[1]

8